In [1]:
import pandas as pd
import numpy as np
import os,sys
os.chdir('C:/Users/arpit.goel/Documents/Projects/Kaggle/27_MCK_Health')
import matplotlib.pyplot as plt
%matplotlib inline

df_train=pd.read_csv('01.RawData/train_ajEneEa.csv')
df_test=pd.read_csv('01.RawData/test_v2akXPA.csv')
df_subm=pd.read_csv('01.RawData/sample_submission_1.csv')

def get_signals(df1):
    df=df1.copy()
    gender_sigs=pd.get_dummies(df['gender']).add_prefix('sig_gender_')
    work_sigs=pd.get_dummies(df['work_type']).add_prefix('sig_work_')
    smoking_sigs=pd.get_dummies(df['smoking_status'].fillna('missing')).add_prefix('sig_smoke_')
    df=pd.concat([df,gender_sigs,work_sigs,smoking_sigs],axis=1)
    for sig in ['age','hypertension','heart_disease','avg_glucose_level','bmi']:
        df['sig_%s'%sig]=df[sig].fillna(-9999)
    df['sig_married']=(df['ever_married']=='Yes').astype(np.int8)
    df['sig_residence']=(df['Residence_type']=='Urban').astype(np.int8)
    df['sig_bmi_missing']=df['bmi'].isnull().astype(np.int8)
    output_sigs=[x for x in df.columns if x[:3] in ['sig','str']]
    return df[output_sigs]

sigs_train=get_signals(df_train)
sigs_test=get_signals(df_test)


In [2]:
bmi_master=pd.concat([sigs_train,sigs_test])
bmi_train=bmi_master[bmi_master['sig_bmi_missing']==0]
bmi_test=bmi_master[bmi_master['sig_bmi_missing']==1]

bmi_train['sample']=(np.random.random(len(bmi_train))<0.8).astype(np.int8)
bmi_test=bmi_train[bmi_train['sample']==0]
bmi_train=bmi_train[bmi_train['sample']==1]

print bmi_train.shape,bmi_test.shape

(47917, 22) (12031, 22)


C:\Users\arpit.goel\AppData\Local\Continuum2\Anaconda2\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [3]:
bmi_idv=[x for x in bmi_train.columns if x[:3] in ['sig'] and 'bmi' not in x]


In [16]:
import xgboost as xgb
from xgboost.sklearn import XGBRegressor
from sklearn.metrics import mean_squared_error,r2_score

def train_xgb(clf):
    clf.fit(bmi_train[bmi_idv], bmi_train['sig_bmi'].values,eval_metric='rmse')
    feat_imp = pd.Series(clf.booster().get_fscore()).sort_values(ascending=False)
    a=mean_squared_error(bmi_train['sig_bmi'],clf.predict(bmi_train[bmi_idv]))
    b=mean_squared_error(bmi_test['sig_bmi'],clf.predict(bmi_test[bmi_idv]))
    c=r2_score(bmi_train['sig_bmi'],clf.predict(bmi_train[bmi_idv]))
    d=r2_score(bmi_test['sig_bmi'],clf.predict(bmi_test[bmi_idv]))
    return a,b,c,d


for param in [50,100,250]:
    xgb = XGBRegressor(n_estimators=50, learning_rate=0.2, gamma=0, subsample=1,
                               colsample_bytree=1, max_depth=3,min_child_weight=1)
    print param,train_xgb(xgb)

50 (41.497844417577653, 43.712884613852871, 0.30790046702516816, 0.2915343825613238)
100 (41.497844417577653, 43.712884613852871, 0.30790046702516816, 0.2915343825613238)
250 (41.497844417577653, 43.712884613852871, 0.30790046702516816, 0.2915343825613238)
